# Setup

In [1]:
import cv2
import cv2.aruco as aruco
import os
import numpy as np
import subprocess

calib_id = "08-05-25"
calib_base_folder = f"/home/arclab/catkin_ws/src/Catheter-Perception/camera_calibration/{calib_id}"
# calib_base_folder = f"C:\\Users\\jlim\\Documents\\GitHub\\Catheter-Perception\\camera_calibration\\{calib_id}"
# Camera 0 refers to top camera, Camera 1 refers to bottom camera
# Ensure camera port ids are correct
# port_ids = [0, 2]
cam0_device = "/dev/cam0"
cam1_device = "/dev/cam1"
cam2_device = "/dev/cam2"
devices = [f"/dev/cam{i}" for i in range(3)]
cap_frame_width = 1280
cap_frame_height = 720

# Ensure proper camera configurations
focus_values = [45, 70, 80]  # Default focus values for cam0, cam1, cam2
config_commands = {cam0_device: [
                    f"v4l2-ctl -d {cam0_device} -c focus_automatic_continuous=0",
                    f"v4l2-ctl -d {cam0_device} -c auto_exposure=3",
                    f"v4l2-ctl -d {cam0_device} -c focus_absolute={focus_values[0]}",
                    # f"v4l2-ctl -d {device} -c exposure_time_absolute=333",
                    # f"v4l2-ctl -d {device} -c gain=0",
                    # f"v4l2-ctl -d {device} -c white_balance_automatic=0",
                    # f"v4l2-ctl -d {device} -c white_balance_temperature=4675",
                    # f"v4l2-ctl -d {device} -c brightness=128",
                    # f"v4l2-ctl -d {device} -c contrast=128",
                    # f"v4l2-ctl -d {device} -c saturation=128",
                    ],
                cam1_device: [
                    f"v4l2-ctl -d {cam1_device} -c focus_automatic_continuous=0",
                    f"v4l2-ctl -d {cam1_device} -c auto_exposure=3",
                    f"v4l2-ctl -d {cam1_device} -c focus_absolute={focus_values[1]}",
                    # f"v4l2-ctl -d {device} -c exposure_time_absolute=333",
                    # f"v4l2-ctl -d {device} -c gain=0",
                    # f"v4l2-ctl -d {device} -c white_balance_automatic=0",
                    # f"v4l2-ctl -d {device} -c white_balance_temperature=4675",
                    # f"v4l2-ctl -d {device} -c brightness=128",
                    # f"v4l2-ctl -d {device} -c contrast=128",
                    # f"v4l2-ctl -d {device} -c saturation=128",
                    ],
                cam2_device: [
                    f"v4l2-ctl -d {cam2_device} -c focus_automatic_continuous=0",
                    f"v4l2-ctl -d {cam2_device} -c auto_exposure=3",
                    f"v4l2-ctl -d {cam2_device} -c focus_absolute={focus_values[2]}",
                    # f"v4l2-ctl -d {device} -c exposure_time_absolute=333",
                    # f"v4l2-ctl -d {device} -c gain=0",
                    # f"v4l2-ctl -d {device} -c white_balance_automatic=0",
                    # f"v4l2-ctl -d {device} -c white_balance_temperature=4675",
                    # f"v4l2-ctl -d {device} -c brightness=128",
                    # f"v4l2-ctl -d {device} -c contrast=128",
                    # f"v4l2-ctl -d {device} -c saturation=128",
                    ],
                }

def configure_camera(devices, config_commands):
    for device in devices:

        print(f"Configuring camera on {device}...")

        for command in config_commands[device]:
            subprocess.run(command, shell=True, check=True)

        print("Camera configuration complete!")

# Find Camera Focus Value

In [ ]:
import subprocess
import cv2
import time

# Configure and open cameras to check proper configs
d=2
cap = cv2.VideoCapture(devices[d], cv2.CAP_V4L2)
cap.set(cv2.CAP_PROP_FRAME_WIDTH, cap_frame_width)
cap.set(cv2.CAP_PROP_FRAME_HEIGHT, cap_frame_height)

if not cap.isOpened():
    print("Error: One or both cameras could not be opened.")
    exit()

focus_adjust = focus_values[d]
configure_wait_time = 2 # wait this many seconds to configure manual settings
configure_yet = False
start_time = time.time()
while True:
    if time.time() - start_time > configure_wait_time and not configure_yet:
        configure_camera([devices[d]], config_commands)
        configure_yet = True
        print("Configured manual camera settings!!!")

    # Read frames from both cameras
    ret, frame = cap.read()

    if not ret:
        print("Error: One or both frames could not be read.")
        break
    cv2.putText(frame, f"Focus: {focus_adjust}", (10, 30), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2)
    cv2.imshow(f"Camera {d}", frame)


    key = cv2.waitKey(1) & 0xFF

    if key == 27:  # ESC key to exit
        break
    elif key == ord('q'):  # Increase focus for Camera 0
        focus_adjust = min(focus_adjust + 5, 250)
        command = f"v4l2-ctl -d {devices[d]} -c focus_absolute={focus_adjust}"
        subprocess.run(command, shell=True)
        print(f"Cam {d} focus set to: {focus_adjust}")
    elif key == ord('a'):  # Decrease focus for Camera 0
        focus_adjust = max(focus_adjust - 5, 0)
        command = f"v4l2-ctl -d {devices[d]} -c focus_absolute={focus_adjust}"
        subprocess.run(command, shell=True)

# Release both cameras and close windows
cap.release()
cv2.destroyAllWindows()

# Check Camera configs

In [ ]:
import subprocess
import cv2
import time

# Configure and open cameras to check proper configs
caps = []
for device in devices:
    cap = cv2.VideoCapture(device, cv2.CAP_V4L2)
    cap.set(cv2.CAP_PROP_FRAME_WIDTH, cap_frame_width)
    cap.set(cv2.CAP_PROP_FRAME_HEIGHT, cap_frame_height)
    caps.append(cap)
    if not cap.isOpened():
        print(f"Error: Camera {device} could not be opened.")
        exit()

configure_wait_time = 2 # wait this many seconds to configure manual settings
configure_yet = False
start_time = time.time()
while True:
    if time.time() - start_time > configure_wait_time and not configure_yet:
        configure_camera(devices, config_commands)
        configure_yet = True
        print("Configured manual camera settings!!!")

    # Read frames from both cameras
    for i, cap in enumerate(caps):
        ret, frame = cap.read()
        if not ret:
            print("Error: Frame could not be read from camera.")
            exit()
        frame = cv2.resize(frame, (cap_frame_width//2, cap_frame_height//2))
        cv2.imshow(f"Camera {i} (Device: {devices[i]})", frame)

    key = cv2.waitKey(1) & 0xFF

    if key == 27:  # ESC key to exit
        break
    elif key == ord('q'):  # Increase focus for Camera 0
        cam0 = min(cam0 + 5, 250)
        command = f"v4l2-ctl -d {cam0_device} -c focus_absolute={cam0_focus_adjust}"
        subprocess.run(command, shell=True)
        print(f"Cam 0 focus set to: {cam0_focus_adjust}")
    elif key == ord('a'):  # Decrease focus for Camera 0
        cam0_focus_adjust = max(cam0_focus_adjust - 5, 0)
        command = f"v4l2-ctl -d {cam0_device} -c focus_absolute={cam0_focus_adjust}"
        subprocess.run(command, shell=True)
        print(f"Cam 0 focus set to: {cam0_focus_adjust}")
    elif key == ord('w'): # Increase focus for Camera 1
        cam1_focus_adjust = min(cam1_focus_adjust + 5, 250)
        command = f"v4l2-ctl -d {cam1_device} -c focus_absolute={cam1_focus_adjust}"
        subprocess.run(command, shell=True)
        print(f"Cam 1 focus set to: {cam1_focus_adjust}")
    elif key == ord('s'):
        # Decrease focus for Camera 1
        cam1_focus_adjust = max(cam1_focus_adjust - 5, 0)
        command = f"v4l2-ctl -d {cam1_device} -c focus_absolute={cam1_focus_adjust}"
        subprocess.run(command, shell=True)
        print(f"Cam 1 focus set to: {cam1_focus_adjust}")

# Release both cameras and close windows
for cap in caps:
    cap.release()
cv2.destroyAllWindows()

# Crop ROI

In [ ]:
import cv2

roi_boxes = []
for d in range(len(devices)):
    cap = cv2.VideoCapture(devices[d], cv2.CAP_V4L2)
    cap.set(cv2.CAP_PROP_FRAME_WIDTH, cap_frame_width)
    cap.set(cv2.CAP_PROP_FRAME_HEIGHT, cap_frame_height)

    print("Press SPACE to freeze frame and select ROI.")
    print("Draw the box, then press ENTER to confirm or ESC to cancel.")

    roi_box = None

    while True:
        ret, frame = cap.read()
        if not ret:
            print("Failed to grab frame.")
            break

        cv2.imshow("Live Feed", frame)
        key = cv2.waitKey(1) & 0xFF

        if key == 32:  # SPACE to pause and select ROI
            cv2.imshow("Select ROI", frame)
            roi_box = cv2.selectROI("Select ROI", frame, showCrosshair=True, fromCenter=False)
            cv2.destroyWindow("Select ROI")

            if roi_box != (0, 0, 0, 0):
                x, y, w, h = roi_box
                cropped = frame[y:y+h, x:x+w]
                cv2.imshow("Cropped ROI", cropped)
                print(f"ROI Coordinates: x={x}, y={y}, w={w}, h={h}")
            else:
                print("No ROI selected.")

        elif key == 27:  # ESC to exit
            break

    cap.release()
    cv2.destroyAllWindows()

    print(f"Final ROI for Camera {d}: {roi_box}")
    roi_boxes.append(roi_box)

# Save the ROI coordinates to a file
roi_file_path = os.path.join(calib_base_folder, f"roi_cam{d}.txt")
with open(roi_file_path, 'w') as f:
    for box in roi_boxes:
        f.write(f"{box[0]} {box[1]} {box[2]} {box[3]}\n")

# Collect Intrinsic Calibration Images

In [ ]:
import cv2
import os
import subprocess
import time

devices = [f"/dev/cam{i}" for i in [2]]

for i, device in enumerate(devices):
    print(f"Collecting instrinsic calibration images for camera {i}...")
    output_dir = f"{calib_base_folder}/intrinsic_calib_images/cam{i}"  # Directory to save images
    os.makedirs(output_dir, exist_ok=True)

    # Initialize webcam
    cap = cv2.VideoCapture(device, cv2.CAP_V4L2)
    cap.set(cv2.CAP_PROP_FRAME_WIDTH, cap_frame_width)
    cap.set(cv2.CAP_PROP_FRAME_HEIGHT, cap_frame_height)

    # Check if camera opened successfully
    if not cap.isOpened():
        print("Error: Could not open camera.")
        exit()

    print("Press SPACE to capture image, ESC to exit.")

    image_count = 0
    configure_wait_time = 2 # wait this many seconds to configure manual settings
    configure_yet = False
    start_time = time.time()
    while True:
        if time.time() - start_time > configure_wait_time and not configure_yet:
            configure_camera([device], config_commands)
            configure_yet = True
            print("Configured manual camera settings!!!")
        
        ret, frame = cap.read()
        if not ret:
            print("Error: Failed to capture image.")
            break

        # Display the live feed
        cv2.imshow("Camera Feed - Press SPACE to capture, ESC to exit", frame)

        # Keyboard input
        key = cv2.waitKey(1) & 0xFF

        # Save image on SPACE key press
        if key == ord(' '):
            # Construct image filename
            image_path = os.path.join(output_dir, f"img_{image_count:03d}.png")
            # Save the frame as PNG
            cv2.imwrite(image_path, frame)
            print(f"Captured: {image_path}")
            image_count += 1

        # Exit on ESC key press
        elif key == 27:
            print("Exiting...")
            break

    # Release the camera and close windows
    cap.release()
    cv2.destroyAllWindows()


# Collect Stereo Calibration Images

In [ ]:
# Collect images of calibration board in both cameras frames for stereo extrinsic calibration
import cv2
import datetime
import os
import time

device_ids = [0, 1]
cur_devices = [f"/dev/cam{i}" for i in device_ids]
output_dir = f"{calib_base_folder}/stereo_calib_images"
os.makedirs(output_dir, exist_ok=True)

caps = []
for device in cur_devices:
    cap = cv2.VideoCapture(device, cv2.CAP_V4L2)
    if not cap.isOpened():
        print(f"Error: Camera {device} could not be opened.")
        exit()
    cap.set(cv2.CAP_PROP_FRAME_WIDTH, cap_frame_width)
    cap.set(cv2.CAP_PROP_FRAME_HEIGHT, cap_frame_height)
    caps.append(cap)

configure_wait_time = 2 # wait this many seconds to configure manual settings
configure_yet = False
start_time = time.time()
frame_count = 0
while True:
    if time.time() - start_time > configure_wait_time and not configure_yet:
        configure_camera(cur_devices, config_commands)
        configure_yet = True
        print("Configured manual camera settings!!!")

    # Read frames from both cameras
    frames = []
    for cap in caps:
        ret, frame = cap.read()
        if not ret:
            print("Error: Frame could not be read from camera.")
            exit()
        # resize for better display
        frame = cv2.resize(frame, (640, 480))
        frames.append(frame)

    # Combine and display both frames
    combined = cv2.hconcat(frames)
    cv2.imshow(f"Camera {device_ids[0]} + Camera {device_ids[1]}", combined)

    key = cv2.waitKey(1) & 0xFF

    if key == 27:  # ESC key to exit
        break
    elif key == ord(' '):  # Space key to capture images
        # Save images with timestamp
        for i, d in enumerate(device_ids):
            img_path = f"{output_dir}/cam{d}_{frame_count}.png"
            cv2.imwrite(img_path, frames[i])
            print(f"Captured images:\n - {img_path}\n")
        frame_count += 1

# Release both cameras and close windows
for cap in caps:
    cap.release()
cv2.destroyAllWindows()

# Collect Test Calib images

In [19]:
# Collect images of calibration board in both cameras frames for stereo extrinsic calibration
import cv2
import datetime
import os
import time

device_ids = [0, 1, 2]
cur_devices = [f"/dev/cam{i}" for i in device_ids]
output_dir = f"{calib_base_folder}/test_calib_images"
os.makedirs(output_dir, exist_ok=True)

caps = []
for device in cur_devices:
    cap = cv2.VideoCapture(device, cv2.CAP_V4L2)
    if not cap.isOpened():
        print(f"Error: Camera {device} could not be opened.")
        exit()
    cap.set(cv2.CAP_PROP_FRAME_WIDTH, cap_frame_width)
    cap.set(cv2.CAP_PROP_FRAME_HEIGHT, cap_frame_height)
    caps.append(cap)

configure_wait_time = 2 # wait this many seconds to configure manual settings
configure_yet = False
start_time = time.time()
frame_count = 0
while True:
    if time.time() - start_time > configure_wait_time and not configure_yet:
        configure_camera(cur_devices, config_commands)
        configure_yet = True
        print("Configured manual camera settings!!!")

    # Read frames from both cameras
    frames = []
    for i, cap in enumerate(caps):
        ret, frame = cap.read()
        if not ret:
            print("Error: Frame could not be read from camera.")
            exit()
        # resize for better display
        frame_display = cv2.resize(frame, (640, 480))
        frames.append(frame)
        cv2.imshow(f"Camera {device_ids[i]}", frame_display)

    # # Combine and display both frames
    # combined = cv2.hconcat(frames)
    # cv2.imshow(f"Camera {device_ids[0]} + Camera {device_ids[1]}", combined)

    key = cv2.waitKey(1) & 0xFF

    if key == 27:  # ESC key to exit
        break
    elif key == ord(' '):  # Space key to capture images
        # Save images with timestamp
        for i, d in enumerate(device_ids):
            img_path = f"{output_dir}/cam{d}_{frame_count}.png"
            cv2.imwrite(img_path, frames[i])
            print(f"Captured images:\n - {img_path}\n")
        frame_count += 1

# Release both cameras and close windows
for cap in caps:
    cap.release()
cv2.destroyAllWindows()

Configuring camera on /dev/cam0...
Camera configuration complete!
Configuring camera on /dev/cam1...
Camera configuration complete!
Configuring camera on /dev/cam2...
Camera configuration complete!
Configured manual camera settings!!!
Captured images:
 - /home/arclab/catkin_ws/src/Catheter-Perception/camera_calibration/08-05-25/test_calib_images/cam0_0.png

Captured images:
 - /home/arclab/catkin_ws/src/Catheter-Perception/camera_calibration/08-05-25/test_calib_images/cam1_0.png

Captured images:
 - /home/arclab/catkin_ws/src/Catheter-Perception/camera_calibration/08-05-25/test_calib_images/cam2_0.png



# Collect Camera-World Calibration Images

In [ ]:
# Collect images for finding camera-world transform
# Current approach is to use aruco marker to find camera 0 pose wrt to aruco marker
# Then camera-camera transform can be used to find camera 1 to world transform
import time

# Make sure cameras are configured
# Open cameras with higher resolution
cap0 = cv2.VideoCapture(cam0_device, cv2.CAP_V4L2)
cap1 = cv2.VideoCapture(cam1_device, cv2.CAP_V4L2)
cap0.set(cv2.CAP_PROP_FRAME_WIDTH, 1280)
cap0.set(cv2.CAP_PROP_FRAME_HEIGHT, 920)
cap1.set(cv2.CAP_PROP_FRAME_WIDTH, 1280)
cap1.set(cv2.CAP_PROP_FRAME_HEIGHT, 920)

if not cap0.isOpened() or not cap1.isOpened():
    print("Error: One or both cameras could not be opened.")
    exit()

output_dir = f"{calib_base_folder}/extrinsic_calib_images"
os.makedirs(output_dir, exist_ok=True)
configure_wait_time = 2 # wait this many seconds to configure manual settings
configure_yet = False
start_time = time.time()
frame_count = 0
while True:

    if time.time() - start_time > configure_wait_time and not configure_yet:
        configure_camera([cam0_device, cam1_device], config_commands)
        configure_yet = True
        print("Configured manual camera settings!!!")

    # Read frame from camera
    ret0, frame0 = cap0.read()
    ret1, frame1 = cap1.read()

    if not ret0 or not ret1:
        print("Error: Frame could not be read.")
        break

    # Combine and display both frames
    combined = cv2.hconcat([frame0, frame1])
    cv2.imshow("Camera 0 (top) + Camera 1 (side)", combined)

    key = cv2.waitKey(1) & 0xFF

    if key == 27:  # ESC key to exit
        break
    elif key == ord(' '):  # Space key to capture images
        # Save images with timestamp
        img0_path = f"{output_dir}/cam0_{frame_count}.png"
        img1_path = f"{output_dir}/cam1_{frame_count}.png"
        cv2.imwrite(img0_path, frame0)
        cv2.imwrite(img1_path, frame1)
        print(f"Captured images:\n - {img0_path}\n - {img1_path}")
        frame_count += 1

# Release both cameras and close windows
cap0.release()
cap1.release()
cv2.destroyAllWindows()

# Collect camera 0 to world calibration image

In [3]:
import cv2
import time
import os
import subprocess

devices = [f"/dev/cam{i}" for i in [0]]

for i, device in enumerate(devices):
    print(f"Collecting world transform calibration images for camera {i}...")
    output_dir = f"{calib_base_folder}/cam{i}_world_transform"  # Directory to save images
    os.makedirs(output_dir, exist_ok=True)

    # Initialize webcam
    cap = cv2.VideoCapture(device, cv2.CAP_V4L2)
    cap.set(cv2.CAP_PROP_FRAME_WIDTH, cap_frame_width)
    cap.set(cv2.CAP_PROP_FRAME_HEIGHT, cap_frame_height)

    # Check if camera opened successfully
    if not cap.isOpened():
        print("Error: Could not open camera.")
        exit()

    print("Press SPACE to capture image, ESC to exit.")

    image_count = 0
    configure_wait_time = 2 # wait this many seconds to configure manual settings
    configure_yet = False
    start_time = time.time()
    while True:
        if time.time() - start_time > configure_wait_time and not configure_yet:
            configure_camera([device], config_commands)
            configure_yet = True
            print("Configured manual camera settings!!!")
        
        ret, frame = cap.read()
        if not ret:
            print("Error: Failed to capture image.")
            break

        # Display the live feed
        cv2.imshow("Camera Feed - Press SPACE to capture, ESC to exit", frame)

        # Keyboard input
        key = cv2.waitKey(1) & 0xFF

        # Save image on SPACE key press
        if key == ord(' '):
            # Construct image filename
            image_path = os.path.join(output_dir, f"img_{image_count:03d}.png")
            # Save the frame as PNG
            cv2.imwrite(image_path, frame)
            print(f"Captured: {image_path}")
            image_count += 1

        # Exit on ESC key press
        elif key == 27:
            print("Exiting...")
            break

    # Release the camera and close windows
    cap.release()
    cv2.destroyAllWindows()


Press SPACE to capture image, ESC to exit.
Configuring camera on /dev/cam0...
Camera configuration complete!
Configured manual camera settings!!!
Captured: /home/arclab/catkin_ws/src/Catheter-Perception/camera_calibration/08-05-25/cam0_world_transform/img_000.png
Exiting...


# Load yaml matrices

In [2]:
import cv2
import numpy as np
import os
import yaml

# Dict holding all camera calibration data
camera_calib_data = {
        "cam0": {
            "intrinsics": {
                "K": None,
                "d": None
            },
            "extrinsics": { # this is camera to world transform
                "R": None,
                "T": None
            }
        },
        "cam1": {
            "intrinsics": {
                "K": None,
                "d": None
            },
            "extrinsics": {
                "R": None,
                "T": None
            }
        },
        "stereo_extrinsics": { # Camera to camera transform (from cam0 to cam1)
            "R_1_0": None,
            "T_1_0": None,
            "R_2_1": None,
            "T_2_1": None
        }, 
        "cam2": {
            "intrinsics": {
                "K": None,
                "d": None
            },
            "extrinsics": {
                "R": None,
                "T": None
            }
        }
    }

def load_kalibr_intrinsics(yaml_data, cam_key):

    cam_data = data[cam_key]
    fx, fy, cx, cy = cam_data['intrinsics']
    K = np.array([[fx, 0, cx],
                  [0, fy, cy],
                  [0,  0,  1]])
    dist = np.array(cam_data['distortion_coeffs'])
    return K, dist

def load_kalibr_extrinsics(yaml_data, cam_key):
    """
    Load extrinsics from kalibr yaml data.
    Returns rotation matrix R and translation vector T.
    """
    cam_data = yaml_data[cam_key]
    matrix = np.array(cam_data['T_cn_cnm1'])
    R = matrix[:3, :3]
    T = matrix[:3, 3].reshape(-1, 1)
    return R, T

# Usage
yaml_path = "/home/arclab/catkin_ws/src/kalibr/08-05-25_T1-camchain.yaml"
with open(yaml_path, 'r') as f:
    data = yaml.safe_load(f)

for i, cam_key in enumerate(['cam0', 'cam1', 'cam2']):
    K, d = load_kalibr_intrinsics(data, cam_key)
    camera_calib_data[cam_key]['intrinsics']['K'] = K
    camera_calib_data[cam_key]['intrinsics']['d'] = d

    # extrinsics
    if i > 0:
        R, T = load_kalibr_extrinsics(data, cam_key)
        camera_calib_data['stereo_extrinsics'][f'R_{i}_{i-1}'] = R
        camera_calib_data['stereo_extrinsics'][f'T_{i}_{i-1}'] = T


print(camera_calib_data)

        


# print("Camera Calibration Data:")
# print(camera_calib_data['cam0']['intrinsics']['K'])


{'cam0': {'intrinsics': {'K': array([[939.00659834,   0.        , 610.34435618],
       [  0.        , 938.71208314, 380.19234786],
       [  0.        ,   0.        ,   1.        ]]), 'd': array([ 0.02409601, -0.11651624,  0.00295776, -0.00425454])}, 'extrinsics': {'R': None, 'T': None}}, 'cam1': {'intrinsics': {'K': array([[946.71215549,   0.        , 635.97087467],
       [  0.        , 947.54229082, 378.04191205],
       [  0.        ,   0.        ,   1.        ]]), 'd': array([ 0.03397451, -0.09771331,  0.00017749, -0.00423783])}, 'extrinsics': {'R': None, 'T': None}}, 'stereo_extrinsics': {'R_1_0': array([[-0.0133914 ,  0.99946859,  0.02971885],
       [ 0.0304311 ,  0.03011512, -0.99908309],
       [-0.99944716, -0.01247474, -0.03081821]]), 'T_1_0': array([[-0.0107076 ],
       [ 0.14082601],
       [ 0.13979356]]), 'R_2_1': array([[-0.02680169,  0.04737187, -0.99851769],
       [ 0.02715598,  0.99854238,  0.04664413],
       [ 0.99927185, -0.02586558, -0.02804905]]), 'T_2_1': a

# Kailbr yaml to py module

In [ ]:
import yaml
import numpy as np
import os

def load_T_cam0_world(path="T_cam0_world.npy"):
    return np.load(path)

def parse_camchain_to_py_module(yaml_path, output_path="camera_calib_data.py", T_cam0_world_path="T_cam0_world.npy"):
    with open(yaml_path, "r") as f:
        data = yaml.safe_load(f)

    if T_cam0_world_path is None:
        T_cam0_world = np.eye(4)
    else:
        T_cam0_world = load_T_cam0_world(T_cam0_world_path)

    output = "import numpy as np\n\n"
    output += "camera_calib_data = {\n"

    cam_keys = sorted([k for k in data.keys() if k.startswith("cam")])
    T_cams_world = {}

    for i, cam_key in enumerate(cam_keys):
        cam = data[cam_key]
        intrinsics = cam["intrinsics"]
        K = np.array([[intrinsics[0], 0, intrinsics[2]],
                      [0, intrinsics[1], intrinsics[3]],
                      [0, 0, 1]])
        d = np.array(cam["distortion_coeffs"]).reshape(1, -1)

        output += f"    '{cam_key}': {{\n"
        output += f"        'intrinsics': {{'K': np.array({repr(K.tolist())}), 'd': np.array({repr(d.tolist())})}},\n"

        if i == 0:
            T_world = T_cam0_world
        else:
            T_rel = np.array(cam["T_cn_cnm1"])
            T_prev = T_cams_world[cam_keys[i - 1]]
            T_world = T_rel @ T_prev

        R = T_world[:3, :3]
        T = T_world[:3, 3].reshape(3, 1)
        T_cams_world[cam_key] = T_world

        output += f"        'extrinsics': {{'R': np.array({repr(R.tolist())}), 'T': np.array({repr(T.tolist())})}},\n"
        output += f"    }},\n"

    # Stereo extrinsics
    T_1_0 = np.array(data['cam1']['T_cn_cnm1'])
    R_1_0 = T_1_0[:3, :3]
    T_1_0_vec = T_1_0[:3, 3].reshape(3, 1)

    output += f"    'stereo_extrinsics': {{'R_1_0': np.array({repr(R_1_0.tolist())}), "
    output += f"'T_1_0': np.array({repr(T_1_0_vec.tolist())})}}\n"
    output += "}\n"

    with open(output_path, "w") as f:
        f.write(output)

    print(f"Saved calibration module to {output_path}")

# Example usage
calib_id = "08-04-25"
calib_base_folder = f"/home/arclab/catkin_ws/src/Catheter-Perception/camera_calibration/{calib_id}"
os.makedirs(calib_base_folder, exist_ok=True)
yaml_path = f"/home/arclab/catkin_ws/src/kalibr/{calib_id}_calib-camchain.yaml"
T_world_path = os.path.join(calib_base_folder, "T_cam0_world.npy")
output_path = os.path.join(calib_base_folder, "camera_calib_data.py")
parse_camchain_to_py_module(yaml_path, output_path, T_world_path)


# Visualize world frame

In [15]:
import numpy as np
import cv2
import os
import matplotlib.pyplot as plt
import importlib.util
# from camera_calib_data import *

def draw_axes(img, K, d, R_wc, T_wc, axis_length=0.05):
    origin = np.array([[0, 0, 0]], dtype=np.float32)
    axes = np.float32([
        [axis_length, 0, 0],
        [0, axis_length, 0],
        [0, 0, axis_length]
    ])
    pts_3d = np.vstack([origin, axes])
    pts_cam = (R_wc @ pts_3d.T + T_wc).T
    pts_2d, _ = cv2.projectPoints(pts_cam, np.zeros(3), np.zeros(3), K, d)
    pts_2d = pts_2d.reshape(-1, 2)

    origin = tuple(pts_2d[0].astype(int))
    x_axis = tuple(pts_2d[1].astype(int))
    y_axis = tuple(pts_2d[2].astype(int))
    z_axis = tuple(pts_2d[3].astype(int))

    cv2.line(img, origin, x_axis, (0, 0, 255), 2)
    cv2.line(img, origin, y_axis, (0, 255, 0), 2)
    cv2.line(img, origin, z_axis, (255, 0, 0), 2)
    return img

def visualize_world_axes_on_images(camera_calib_data, image_root=".", save_folder=None):

    for cam_key in ['cam0', 'cam1', 'cam2']:
        K = camera_calib_data[cam_key]['intrinsics']['K']
        d = camera_calib_data[cam_key]['intrinsics']['d']
        R = camera_calib_data[cam_key]['extrinsics']['R']
        T = camera_calib_data[cam_key]['extrinsics']['T']
        
        rvec = cv2.Rodrigues(R)[0]
        tvec = T.reshape(3, 1)

        # for i in range(3):
        img_path = os.path.join(image_root, f"{cam_key}_0.png")
        img = cv2.imread(img_path)
        if img is None:
            print(f"Could not load {img_path}")
            continue
        img_axes = cv2.drawFrameAxes(img, K, d, rvec, tvec, length=0.05, thickness=2)
        cv2.imshow(f"{cam_key} World Axes", img_axes)
    cv2.waitKey(0)
    cv2.destroyAllWindows()
        # save_path = os.path.join(save_folder, f"{cam_key}_world_axes.png")
        # cv2.imwrite(save_path, img_axes)
        # print(f"Saved {save_path}")

# Run the visualization
visualize_world_axes_on_images(
    camera_calib_data,
    image_root="/home/arclab/catkin_ws/src/Catheter-Perception/camera_calibration/08-05-25/test_calib_images",
)

In [ ]:
import cv2
import numpy as np
import matplotlib.pyplot as plt
import importlib.util

def load_camera_calib_data(module_path):
    spec = importlib.util.spec_from_file_location("camera_calib_data", module_path)
    calib_module = importlib.util.module_from_spec(spec)
    spec.loader.exec_module(calib_module)
    return calib_module.camera_calib_data

def draw_axes_on_image(img, K, R, T, axis_length=0.05):
    """
    Draws the 3D world coordinate axes on the 2D image using the given camera intrinsics and extrinsics.

    Args:
        img (np.ndarray): The image to draw on.
        K (np.ndarray): Camera intrinsics (3x3).
        R (np.ndarray): Camera rotation matrix (3x3).
        T (np.ndarray): Camera translation vector (3x1).
        axis_length (float): Length of the axes in meters.

    Returns:
        np.ndarray: Image with axes drawn.
    """
    # Define the 3D points in world frame
    origin = np.zeros((3, 1))
    x_axis = np.array([[axis_length, 0, 0]]).T
    y_axis = np.array([[0, axis_length, 0]]).T
    z_axis = np.array([[0, 0, axis_length]]).T

    points_3D = np.hstack([origin, x_axis, y_axis, z_axis])  # shape (3, 4)

    # Project to image using cv2.projectPoints (expects rvec, tvec)
    rvec, _ = cv2.Rodrigues(R)
    tvec = T

    points_2D, _ = cv2.projectPoints(points_3D.T, rvec, tvec, K, None)
    points_2D = points_2D.squeeze().astype(int)

    origin_2D = tuple(points_2D[0])
    x_2D = tuple(points_2D[1])
    y_2D = tuple(points_2D[2])
    z_2D = tuple(points_2D[3])

    # Draw the axes
    img = cv2.line(img, origin_2D, x_2D, (0, 0, 255), 2)  # X - red
    img = cv2.line(img, origin_2D, y_2D, (0, 255, 0), 2)  # Y - green
    img = cv2.line(img, origin_2D, z_2D, (255, 0, 0), 2)  # Z - blue

    return img

def visualize_axes_on_all_cameras(images_dict, camera_calib_data):
    """
    Given a dict of images for each camera, overlay the world axes on each.

    Args:
        images_dict (dict): Dictionary of images with keys 'cam0', 'cam1', 'cam2'
    """
    for cam_id in ['cam0', 'cam1', 'cam2']:
        calib = camera_calib_data[cam_id]
        K = calib['intrinsics']['K']
        R = calib['extrinsics']['R']
        T = calib['extrinsics']['T']

        img = images_dict[cam_id].copy()
        img_with_axes = draw_axes_on_image(img, K, R, T)

        cv2.imshow(cam_id, img_with_axes)

    print("Press any key to close...")
    cv2.waitKey(0)
    cv2.destroyAllWindows()

# Example usage
if __name__ == "__main__":
    # Load or capture an image for each camera
    # Replace these with actual image frames from your cameras
    root_dir = "/home/arclab/catkin_ws/src/Catheter-Perception/camera_calibration/08-04-25"
    img0 = cv2.imread(f"{root_dir}/cam0/img_000.png")
    img1 = cv2.imread(f"{root_dir}/cam1/img_000.png")
    img2 = cv2.imread(f"{root_dir}/cam2/img_000.png")
    calib_module_path="/home/arclab/catkin_ws/src/Catheter-Perception/camera_calibration/08-04-25/camera_calib_data.py"
    camera_calib_data = load_camera_calib_data(calib_module_path)

    images = {'cam0': img0, 'cam1': img1, 'cam2': img2}
    visualize_axes_on_all_cameras(images, camera_calib_data)

# Compute Camera Calibration

## Compute Intrinsics Charuco

In [ ]:
# Compute intrinsic, stereo extrinsic, and camera to world transforms
import cv2
import numpy as np
import glob
import cv2.aruco as aruco
import os
import pickle

def calibrate_intrinsics_charuco(glob_pattern, charuco_board, aruco_dict, min_markers=20):
    '''
    Calibrates camera intrinsics using a ChArUco board.
    '''
    all_corners, all_ids, img_size = [], [], None

    for fname in glob.glob(glob_pattern):
        img = cv2.imread(fname)
        img_gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
        if img_size is None:
            img_size = img_gray.shape[::-1]

        detector = aruco.CharucoDetector(charuco_board)
        charuco_corners, charuco_ids, marker_corners, marker_ids = detector.detectBoard(img_gray)

        # ⛔️ Skip images with insufficient corner detections
        if charuco_corners is None or charuco_ids is None or len(charuco_corners) < min_markers:
            print(f"Skipping {fname} - Not enough corners detected.")
            continue

        all_corners.append(charuco_corners)
        all_ids.append(charuco_ids)

    # ✅ Only calibrate if you have enough valid detections
    if len(all_corners) == 0:
        raise ValueError("No valid ChArUco detections. Check your images or board setup.")

    # Calibrate
    ret, K, dist, rvecs, tvecs = aruco.calibrateCameraCharuco(
        charucoCorners=all_corners,
        charucoIds=all_ids,
        board=charuco_board,
        imageSize=img_size,
        cameraMatrix=None,
        distCoeffs=None
    )

    return ret, K, dist, rvecs, tvecs


# Dict holding all camera calibration data
camera_calib_data = {
        "cam0": {
            "intrinsics": {
                "K": None,
                "d": None
            },
            "extrinsics": { # this is camera to world transform
                "R": None,
                "T": None
            }
        },
        "cam1": {
            "intrinsics": {
                "K": None,
                "d": None
            },
            "extrinsics": {
                "R": None,
                "T": None
            }
        },
        "stereo_extrinsics": { # Camera to camera transform (from cam0 to cam1)
            "R_1_0": None,
            "T_1_0": None,
            "R_2_0": None,
            "T_2_0": None
        }, 
        "cam2": {
            "intrinsics": {
                "K": None,
                "d": None
            },
            "extrinsics": {
                "R": None,
                "T": None
            }
        }
    }

# Define ChArUco board used
square_size = 0.010
marker_length = 0.006
board_size = (10, 10)
aruco_dict = aruco.getPredefinedDictionary(aruco.DICT_4X4_50)
charuco_board_A = aruco.CharucoBoard(
    size=board_size,
    squareLength=square_size, 
    markerLength=marker_length,
    dictionary=aruco_dict
)

charuco_board_B = aruco.CharucoBoard(
    size=(10, 10),
    squareLength=0.006, 
    markerLength=0.004,
    dictionary=aruco_dict
)

# Calibrate instrinsics
for i in range(3):
    charuco_board = charuco_board_A if i < 2 else charuco_board_B
    intrinsic_calib_images_path = f"{calib_base_folder}/intrinsic_calib_images/cam{i}/*.png"
    rms_error, K, d, _, _ = calibrate_intrinsics_charuco(
        intrinsic_calib_images_path, charuco_board, aruco_dict
    )
    print(f"Charuco intrinsics for camera {i} RMS error: {rms_error:.4f}")
    print(f"Camera {i} intrinsic matrix:\n{K}")
    print(f"Camera {i} distortion coefficients:\n{d}")
    camera_calib_data[f"cam{i}"]["intrinsics"]["K"] = K
    camera_calib_data[f"cam{i}"]["intrinsics"]["d"] = d

print("Intrinsic calibration complete.")


## Compute Stereo Extrinsics Charuco

In [ ]:
import cv2
import numpy as np
import glob
import cv2.aruco as aruco
import os
import pickle


def detect_charuco_corners(image_path, board, detector):
    """Detects Charuco corners using OpenCV 4.11+ functions."""
    img = cv2.imread(image_path)
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)

    # Detect Aruco markers and Charuco corners
    charuco_corners, charuco_ids, _, _ = detector.detectBoard(gray)

    if charuco_ids is not None and len(charuco_ids) > 4:
        return charuco_corners, charuco_ids
    return None, None

def calculate_relative_transform(R1, T1, R2, T2):
    """Calculate the relative rotation and translation between two cameras.
    Returns R21 and T21, the rotation and translation of camera 2 in camera 1's frame."""
    R21 = np.dot(R2, R1.T)
    T21 = T2 - np.dot(R21, T1)
    return R21, T21

def get_extrinsics(image_path, K, dist, board, detector):
    """Compute extrinsics for a single image."""
    charuco_corners, charuco_ids = detect_charuco_corners(image_path, board, detector)
    if charuco_corners is not None:
        obj_points = board.getChessboardCorners()[charuco_ids.flatten()]
        ret, rvec, tvec = cv2.solvePnP(obj_points, charuco_corners, K, dist)
        if ret:
            R, _ = cv2.Rodrigues(rvec)
            return R, tvec
    return None, None

# # Load intrinsic parameters
# calib_data_file = "../camera_calibration/05-08-25/camera_calib_data.pkl"
# with open(calib_data_file, 'rb') as f:
#     calib_data = pickle.load(f)

K0 = camera_calib_data["cam0"]["intrinsics"]["K"]
dist0 = camera_calib_data["cam0"]["intrinsics"]["d"]
K1 = camera_calib_data["cam1"]["intrinsics"]["K"]
dist1 = camera_calib_data["cam1"]["intrinsics"]["d"]

# Define ChArUco board used
square_size = 0.006
marker_length = 0.004
board_size = (10, 10)
aruco_dict = aruco.getPredefinedDictionary(aruco.DICT_4X4_50)
charuco_board = aruco.CharucoBoard(
    size=board_size,
    squareLength=square_size, 
    markerLength=marker_length,
    dictionary=aruco_dict
)

# Create the Aruco and Charuco detector objects (new in OpenCV 4.11+)
aruco_detector = aruco.ArucoDetector(aruco_dict)
charuco_detector = aruco.CharucoDetector(charuco_board)

# Paths to synchronized images
cam0_images = sorted(glob.glob(f"{calib_base_folder}/stereo_calib_images/cams_0_1/cam0_*.png"))
cam1_images = sorted(glob.glob(f"{calib_base_folder}/stereo_calib_images/cams_0_1/cam1_*.png"))

# Arrays to store transformations
relative_rotations = []
relative_translations = []

for img0_path, img1_path in zip(cam0_images, cam1_images):
    # Get extrinsics for both cameras using updated functions
    R0, T0 = get_extrinsics(img0_path, K0, dist0, charuco_board, charuco_detector)
    R1, T1 = get_extrinsics(img1_path, K1, dist1, charuco_board, charuco_detector)

    if R0 is not None and R1 is not None:
        # Calculate the relative transformation between cameras
        R10, T10 = calculate_relative_transform(R0, T0, R1, T1)
        relative_rotations.append(R10)
        relative_translations.append(T10)

# Average the transformations
R_avg = sum(relative_rotations) / len(relative_rotations)
T_avg = sum(relative_translations) / len(relative_translations)

# Print the averaged relative transformation
print("Transform from Camera 0 to Camera 1:")
print("Averaged Relative Rotation (R10):\n", R_avg)
print("Averaged Relative Translation (T10):\n", T_avg)

# Save the relative transformation
camera_calib_data["stereo_extrinsics"]["R_1_0"] = R_avg
camera_calib_data["stereo_extrinsics"]["T_1_0"] = T_avg


In [ ]:
import cv2
import numpy as np
import glob
import cv2.aruco as aruco
import os
import pickle


def detect_charuco_corners(image_path, board, detector):
    """Detects Charuco corners using OpenCV 4.11+ functions."""
    img = cv2.imread(image_path)
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)

    # Detect Aruco markers and Charuco corners
    charuco_corners, charuco_ids, _, _ = detector.detectBoard(gray)

    if charuco_ids is not None and len(charuco_ids) > 4:
        return charuco_corners, charuco_ids
    return None, None

def calculate_relative_transform(R1, T1, R2, T2):
    """Calculate the relative rotation and translation between two cameras.
    Returns R21 and T21, the rotation and translation of camera 2 in camera 1's frame."""
    R21 = np.dot(R2, R1.T)
    T21 = T2 - np.dot(R21, T1)
    return R21, T21

def get_extrinsics(image_path, K, dist, board, detector):
    """Compute extrinsics for a single image."""
    charuco_corners, charuco_ids = detect_charuco_corners(image_path, board, detector)
    if charuco_corners is not None:
        obj_points = board.getChessboardCorners()[charuco_ids.flatten()]
        ret, rvec, tvec = cv2.solvePnP(obj_points, charuco_corners, K, dist)
        if ret:
            R, _ = cv2.Rodrigues(rvec)
            return R, tvec
    return None, None

# # Load intrinsic parameters
# calib_data_file = "../camera_calibration/05-08-25/camera_calib_data.pkl"
# with open(calib_data_file, 'rb') as f:
#     calib_data = pickle.load(f)

K0 = camera_calib_data["cam0"]["intrinsics"]["K"]
dist0 = camera_calib_data["cam0"]["intrinsics"]["d"]
K2 = camera_calib_data["cam2"]["intrinsics"]["K"]
dist2 = camera_calib_data["cam2"]["intrinsics"]["d"]

# Define ChArUco board used
square_size = 0.006
marker_length = 0.004
board_size = (10, 10)
aruco_dict = aruco.getPredefinedDictionary(aruco.DICT_4X4_50)
charuco_board = aruco.CharucoBoard(
    size=board_size,
    squareLength=square_size, 
    markerLength=marker_length,
    dictionary=aruco_dict
)

# Create the Aruco and Charuco detector objects (new in OpenCV 4.11+)
aruco_detector = aruco.ArucoDetector(aruco_dict)
charuco_detector = aruco.CharucoDetector(charuco_board)

# Paths to synchronized images
cam0_images = sorted(glob.glob(f"{calib_base_folder}/stereo_calib_images/cams_0_2/cam0_*.png"))
cam1_images = sorted(glob.glob(f"{calib_base_folder}/stereo_calib_images/cams_0_2/cam2_*.png"))

# Arrays to store transformations
relative_rotations = []
relative_translations = []

for img0_path, img1_path in zip(cam0_images, cam1_images):
    # Get extrinsics for both cameras using updated functions
    R0, T0 = get_extrinsics(img0_path, K0, dist0, charuco_board, charuco_detector)
    R1, T1 = get_extrinsics(img1_path, K1, dist1, charuco_board, charuco_detector)

    if R0 is not None and R1 is not None:
        # Calculate the relative transformation between cameras
        R10, T10 = calculate_relative_transform(R0, T0, R1, T1)
        relative_rotations.append(R10)
        relative_translations.append(T10)

# Average the transformations
R_avg = sum(relative_rotations) / len(relative_rotations)
T_avg = sum(relative_translations) / len(relative_translations)

# Print the averaged relative transformation
print("Transform from Camera 0 to Camera 2:")
print("Averaged Relative Rotation (R10):\n", R_avg)
print("Averaged Relative Translation (T10):\n", T_avg)

# Save the relative transformation
camera_calib_data["stereo_extrinsics"]["R_2_0"] = R_avg
camera_calib_data["stereo_extrinsics"]["T_2_0"] = T_avg


# Compute World Transform Charuco Board

In [17]:
import cv2
import numpy as np
import cv2.aruco as aruco
import os
import pickle

def compute_camera_to_world_transform(image_path, cam_key, camera_calib_data, charuco_board, origin_idx=0, visualize=True):
    """
    Estimates the camera-to-world transform using a ChArUco board, and draws world frame axes at a specified corner.
    
    Args:
        image_path (str): Path to the image showing the ChArUco board.
        cam_key (str): 'cam0', 'cam1', or 'cam2'
        camera_calib_data (dict): Dictionary holding K, d, and will be updated with R, T.
        charuco_board (cv2.aruco.CharucoBoard): The ChArUco board definition.
        origin_idx (int): Index of the ChArUco corner to use as the world origin.
        visualize (bool): Whether to display image with drawn corners and axes.
    """

    # === Get intrinsics ===
    K = camera_calib_data[cam_key]["intrinsics"]["K"]
    d = camera_calib_data[cam_key]["intrinsics"]["d"]
    if K is None or d is None:
        raise ValueError(f"Missing intrinsics for {cam_key}")

    # === Read image ===
    img = cv2.imread(image_path)
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)

    # === Detect board ===
    detector = aruco.CharucoDetector(charuco_board)
    charuco_corners, charuco_ids, marker_corners, marker_ids = detector.detectBoard(gray)

    if charuco_corners is None or charuco_ids is None or len(charuco_corners) < 4:
        raise RuntimeError("ChArUco board detection failed or not enough corners.")

    # === Estimate pose of board ===
    retval, rvec, tvec = aruco.estimatePoseCharucoBoard(
        charuco_corners, charuco_ids, charuco_board, K, d, None, None
    )
    if not retval:
        raise RuntimeError("Pose estimation failed.")

    # === Convert rotation vector to matrix ===
    R, _ = cv2.Rodrigues(rvec)
    T = tvec.reshape((3, 1))

    # === Adjust world origin if origin_idx is visible ===
    detected_ids = charuco_ids.flatten()
    if origin_idx in detected_ids:
        origin_corner_in_board = charuco_board.getChessboardCorners()[origin_idx].reshape(3, 1)
        print(origin_corner_in_board)
        T = T + R @ origin_corner_in_board
        adjusted_tvec = tvec + R @ origin_corner_in_board
    else:
        print(f"Warning: origin_idx {origin_idx} not found in detected IDs. Using default origin.")
        adjusted_tvec = tvec

    # === Store extrinsics into the camera_calib_data dict ===
    camera_calib_data[cam_key]["extrinsics"]["R"] = R
    camera_calib_data[cam_key]["extrinsics"]["T"] = T

    print(f"Camera {cam_key} extrinsic calibration complete.")
    print("R = \n", R)
    print("T = \n", T)

    # === Visualization ===
    if visualize:
        img_display = img.copy()
        aruco.drawDetectedCornersCharuco(img_display, charuco_corners, charuco_ids, (0, 255, 0))

        # Draw frame axes at the new origin (charuco corner specified by origin_idx)
        axis_length = 0.03  # 3 cm
        cv2.drawFrameAxes(img_display, K, d, rvec, adjusted_tvec, axis_length)

        cv2.imshow(f"{cam_key} - ChArUco + Axes at corner {origin_idx}", img_display)
        cv2.waitKey(0)
        cv2.destroyAllWindows()

    return R, T


image_path = f"{calib_base_folder}/cam0_world_transform/img_000.png"
charuco_board = aruco.CharucoBoard(
    size=(10, 10),
    squareLength=0.010,
    markerLength=0.006,
    dictionary=aruco.getPredefinedDictionary(aruco.DICT_4X4_50)
)
# Compute camera to world transform for cam0
R_0_w, T_0_w = compute_camera_to_world_transform(
    image_path, "cam0", camera_calib_data, charuco_board, origin_idx=67
)

# Chain transforms to get world transforms for cams 1 and 2
R_1_0 = camera_calib_data["stereo_extrinsics"]["R_1_0"]
T_1_0 = camera_calib_data["stereo_extrinsics"]["T_1_0"]
R_2_1 = camera_calib_data["stereo_extrinsics"]["R_2_1"]
T_2_1 = camera_calib_data["stereo_extrinsics"]["T_2_1"]
R_1_w = R_1_0 @ R_0_w
T_1_w = R_1_0 @ T_0_w + T_1_0
R_2_w = R_2_1 @ R_1_0 @ R_0_w
T_2_0 = R_2_1 @ T_1_0 + T_2_1
T_2_w =  R_2_1 @ T_1_w + T_2_1
# Store the transforms
camera_calib_data["cam1"]["extrinsics"]["R"] = R_1_w
camera_calib_data["cam1"]["extrinsics"]["T"] = T_1_w
camera_calib_data["cam2"]["extrinsics"]["R"] = R_2_w
camera_calib_data["cam2"]["extrinsics"]["T"] = T_2_w

print(camera_calib_data)

# Save the camera calibration data to a file
with open(f"{calib_base_folder}/camera_calib_data.pkl", 'wb') as f:
    pickle.dump(camera_calib_data, f)

[[0.05]
 [0.08]
 [0.  ]]
Camera cam0 extrinsic calibration complete.
R = 
 [[ 0.02125371  0.99865781 -0.04723202]
 [-0.99973489  0.02164761  0.00784379]
 [ 0.00885572  0.04705279  0.99885315]]
T = 
 [[-0.00252137]
 [-0.00207384]
 [ 0.15525986]]
{'cam0': {'intrinsics': {'K': array([[939.00659834,   0.        , 610.34435618],
       [  0.        , 938.71208314, 380.19234786],
       [  0.        ,   0.        ,   1.        ]]), 'd': array([ 0.02409601, -0.11651624,  0.00295776, -0.00425454])}, 'extrinsics': {'R': array([[ 0.02125371,  0.99865781, -0.04723202],
       [-0.99973489,  0.02164761,  0.00784379],
       [ 0.00885572,  0.04705279,  0.99885315]]), 'T': array([[-0.00252137],
       [-0.00207384],
       [ 0.15525986]])}}, 'cam1': {'intrinsics': {'K': array([[946.71215549,   0.        , 635.97087467],
       [  0.        , 947.54229082, 378.04191205],
       [  0.        ,   0.        ,   1.        ]]), 'd': array([ 0.03397451, -0.09771331,  0.00017749, -0.00423783])}, 'extrinsics

## Compute Camera-World Transform

In [ ]:
# Find camera to world using PnP and calibrator piece
import cv2
import os

img_dir = "../camera_calibration/05-16-25/extrinsic_calib_images"
img0 = cv2.imread(os.path.join(img_dir, "cam0_0.png"))
img1 = cv2.imread(os.path.join(img_dir, "cam1_0.png"))
# Downsize images for display
img0 = cv2.resize(img0, (640, 480))
img1 = cv2.resize(img1, (640, 480))


In [ ]:

# Lists to store clicked points
points_img0 = []
points_img1 = []

def click_event_img0(event, x, y, flags, param):
    if event == cv2.EVENT_LBUTTONDOWN:
        points_img0.append((x, y))
        print(f"Image 0: Point {len(points_img0)}: ({x}, {y})")
        cv2.circle(img0, (x, y), 5, (0, 0, 255), -1)
        cv2.imshow("Cam0", img0)

def click_event_img1(event, x, y, flags, param):
    if event == cv2.EVENT_LBUTTONDOWN:
        points_img1.append((x, y))
        print(f"Image 1: Point {len(points_img1)}: ({x}, {y})")
        cv2.circle(img1, (x, y), 5, (0, 0, 255), -1)
        cv2.imshow("Cam1", img1)

cv2.imshow("Cam0", img0)
cv2.setMouseCallback("Cam0", click_event_img0)
cv2.imshow("Cam1", img1)
cv2.setMouseCallback("Cam1", click_event_img1)

print("Click points in both images. Press any key to finish and close windows.")
cv2.waitKey(0)
cv2.destroyAllWindows()

print("Points in img0:", points_img0)
print("Points in img1:", points_img1)


In [ ]:
# Read in camera intrinsics
with open("../camera_calibration/05-08-25/camera_calib_data.pkl", 'rb') as f:
    camera_calib_data = pickle.load(f)
K0 = camera_calib_data["cam0"]["intrinsics"]["K"]
dist0 = camera_calib_data["cam0"]["intrinsics"]["d"]
K1 = camera_calib_data["cam1"]["intrinsics"]["K"]
dist1 = camera_calib_data["cam1"]["intrinsics"]["d"]

# Defined corresponding world frame coordinates of clicked points
# These should be in the same order as the clicked points
world_coords_0 = np.array([[0, 4.125, 0],
                [0, 16, 9],
                [0, 0, 23.95],
                [-4.125, 0, 0],
                [-16, 0, 9],
                [4.125, 0, 0],
                [16, 0, 9]]) / 1000

world_coords_1 = np.array([[0, 4.125, 0],
                [0, 16, 9],
                [0, 0, 23.95],
                [-4.125, 0, 0],
                [-16, 0, 9],
                [0, -4.125, 0],
                [0, -16, 9]]) / 1000

# Find world frame transform for each camera using solvePnP
def find_camera_to_world_transform(points_img, world_coords, K, dist):
    # Convert points to numpy arrays
    points_img = np.array(points_img, dtype=np.float32)
    world_coords = np.array(world_coords, dtype=np.float32)

    # Solve PnP
    _, rvec, tvec = cv2.solvePnP(world_coords, points_img, K, dist)
    R, _ = cv2.Rodrigues(rvec)
    return R, tvec

R0, T0 = find_camera_to_world_transform(points_img0, world_coords_0, K0, dist0)
R1, T1 = find_camera_to_world_transform(points_img1, world_coords_1, K1, dist1)
print("Camera 0 to world transform:")
print("Rotation:\n", R0)
print("Translation:\n", T0)
print("Camera 1 to world transform:")
print("Rotation:\n", R1)
print("Translation:\n", T1)

# Visualize world frame axes in each camera frame
def draw_axes(image, R, T, K, dist, axis_length=0.02):
    """Draw the world frame axes on the image."""
    axis_points = np.float32([
        [0, 0, 0],                  # Origin
        [axis_length, 0, 0],        # X-axis (red)
        [0, axis_length, 0],        # Y-axis (green)
        [0, 0, axis_length]         # Z-axis (blue)
    ]).reshape(-1, 3)
    img_points, _ = cv2.projectPoints(axis_points, cv2.Rodrigues(R)[0], T, K, dist)
    origin = tuple(img_points[0].ravel().astype(int))
    cv2.line(image, origin, tuple(img_points[1].ravel().astype(int)), (0, 0, 255), 3)
    cv2.line(image, origin, tuple(img_points[2].ravel().astype(int)), (0, 255, 0), 3)
    cv2.line(image, origin, tuple(img_points[3].ravel().astype(int)), (255, 0, 0), 3)
    cv2.putText(image, "X", tuple(img_points[1].ravel().astype(int)), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 255), 2)
    cv2.putText(image, "Y", tuple(img_points[2].ravel().astype(int)), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2)
    cv2.putText(image, "Z", tuple(img_points[3].ravel().astype(int)), cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 0, 0), 2)


# # Replace extrinsic calibration parameters
# camera_calib_data["cam0"]["extrinsics"]["R"] = R0
# camera_calib_data["cam0"]["extrinsics"]["T"] = T0
# camera_calib_data["cam1"]["extrinsics"]["R"] = R1
# camera_calib_data["cam1"]["extrinsics"]["T"] = T1
# # Save the camera calibration data
# with open(f"../camera_calibration/05-16-25/camera_calib_data.pkl", 'wb') as f:
#     pickle.dump(camera_calib_data, f)

# Show camera 0 world frame transform in both camera frames
# Compute Camera 1 to world transform using stereo extrinsics
R_1_0 = camera_calib_data["stereo_extrinsics"]["R_1_0"]
T_1_0 = camera_calib_data["stereo_extrinsics"]["T_1_0"]
R1_from0 = np.dot(R_1_0, R0)
T1_from0 = np.dot(R_1_0, T0) + T_1_0
print("Camera 1 to world transform using stereo extrinsics:")
print("Rotation:\n", R1_from0)
print("Translation:\n", T1_from0)
# Show camera 1 world frame transform in both camera frames
R0_from1 = np.dot(R_1_0.T, R1)
T0_from1 = np.dot(R_1_0.T, T1) - T_1_0
print("Camera 0 to world transform using stereo extrinsics:")
print("Rotation:\n", R0_from1)
print("Translation:\n", T0_from1)

# Draw axes on img0 and img1
# draw_axes(img0, R0, T0, K0, dist0)
draw_axes(img0, R0_from1, T0_from1, K0, dist0)
cv2.imshow("Camera 0 - World Frame Axes from cam 0 transform", img0)
# cv2.imshow("Camera 1 - World Frame Axes from cam 0 transform", img1)
cv2.waitKey(0)
cv2.destroyAllWindows()

# Draw axes on img0 and img1
# draw_axes(img1, R1_from0, T1_from0, K1, dist1)
draw_axes(img1, R1, T1, K1, dist1)
# cv2.imshow("Camera 0 - World Frame Axes from cam 1 transform", img0)
cv2.imshow("Camera 1 - World Frame Axes from cam 1 transform", img1)
cv2.waitKey(0)
cv2.destroyAllWindows()

In [ ]:
import numpy as np

def transform_matrix(R, t):
    """Construct a homogeneous transformation matrix from rotation and translation."""
    T = np.eye(4)
    T[:3, :3] = R
    T[:3, 3] = t
    return T

def check_transform_equivalence(R1, t1, R2, t2):
    """Check if two transformations are equivalent."""
    # Construct transformation matrices
    T1 = transform_matrix(R1, t1)
    T2 = transform_matrix(R2, t2)
    
    # Compute the relative transformation
    relative_transform = np.linalg.inv(T1) @ T2
    identity_matrix = np.eye(4)
    
    # Compute the deviation from the identity matrix
    deviation = np.linalg.norm(relative_transform - identity_matrix)

    # Print the relative transformation and the deviation
    print("Relative Transformation:\n", relative_transform)
    print("Deviation from Identity:", deviation)

    # Threshold for considering them equivalent
    tolerance = 1e-6
    if deviation < tolerance:
        print("The transformations are approximately equivalent.")
    else:
        print("The transformations are not equivalent.")

# # Example usage
# R1 = np.array([[-0.996, -0.087, 0.012],
#                [-0.032, 0.234, -0.972],
#                [0.082, -0.968, -0.236]])

# t1 = np.array([-0.003, 0.045, -0.221])

# R2 = np.array([[-0.104, -0.994, 0.029],
#                [-0.080, -0.020, -0.997],
#                [0.991, -0.106, -0.078]])

# t2 = np.array([0.015, 0.028, -0.273])

check_transform_equivalence(R1, T0.flatten(), R0_from1, T0_from1.flatten())
check_transform_equivalence(R1, T1.flatten(), R1_from0, T1_from0.flatten())


In [ ]:
import numpy as np
from scipy.optimize import least_squares
import pdb

### Currently does not work

def rotation_matrix_to_vector(R):
    """Convert a rotation matrix to a rotation vector using Rodrigues' formula."""
    rvec, _ = cv2.Rodrigues(R)
    return rvec.flatten()


def vector_to_rotation_matrix(rvec):
    """Convert a rotation vector to a rotation matrix using Rodrigues' formula."""
    R, _ = cv2.Rodrigues(rvec)
    return R


def transform_matrix(R, t):
    """Construct a homogeneous transformation matrix from rotation and translation."""
    T = np.eye(4)
    T[:3, :3] = R
    T[:3, 3] = t
    return T


def decompose_transform(T):
    """Decompose a transformation matrix into rotation and translation."""
    R = T[:3, :3]
    t = T[:3, 3]
    return R, t


def cost_function(params, R_0_w, T_0_W, R_1_0, T_1_0, R_1_w, T_1_W):
    """Cost function to minimize the difference between expected and actual transforms."""
    rvec0, rvec1, t0, t1 = np.split(params, [3, 6, 9])
    R0 = vector_to_rotation_matrix(rvec0)
    R1 = vector_to_rotation_matrix(rvec1)
    T0 = transform_matrix(R0, t0)
    T1 = transform_matrix(R1, t1)
    expected_T1 = T0 @ transform_matrix(R_1_0, T_1_0)
    residual = (T1 - expected_T1).flatten()
    return residual


def refine_transforms(R_0_w, T_0_W, R_1_0, T_1_0, R_1_w, T_1_W):
    """Optimize transforms to improve consistency."""
    rvec0 = rotation_matrix_to_vector(R_0_w)
    rvec1 = rotation_matrix_to_vector(R_1_w)
    # pdb.set_trace()
    initial_params = np.hstack([rvec0, rvec1, T_0_W, T_1_W])
    result = least_squares(cost_function, initial_params, args=(R_0_w, T_0_W, R_1_0, T_1_0, R_1_w, T_1_W))
    rvec0_opt, rvec1_opt, t0_opt, t1_opt = np.split(result.x, [3, 6, 9])
    R0_opt = vector_to_rotation_matrix(rvec0_opt)
    R1_opt = vector_to_rotation_matrix(rvec1_opt)
    return R0_opt, t0_opt, R1_opt, t1_opt

# Load the camera calibration data
with open(f"../camera_calibration/05-16-25/camera_calib_data.pkl", 'rb') as f:
    camera_calib_data = pickle.load(f)

R_0_w = camera_calib_data["cam0"]["extrinsics"]["R"]
T_0_W = camera_calib_data["cam0"]["extrinsics"]["T"].flatten()
R_1_0 = camera_calib_data["stereo_extrinsics"]["R_1_0"]
T_1_0 = camera_calib_data["stereo_extrinsics"]["T_1_0"].flatten()
R_1_w = camera_calib_data["cam1"]["extrinsics"]["R"]
T_1_W = camera_calib_data["cam1"]["extrinsics"]["T"].flatten()
# Refine the transforms
R0_refined, T0_refined, R1_refined, T1_refined = refine_transforms(R_0_w, T_0_W, R_1_0, T_1_0, R_1_w, T_1_W)
print("Refined R_0_w:\n", R0_refined)
print("Refined T_0_W:\n", T0_refined)
print("Refined R_1_w:\n", R1_refined)
print("Refined T_1_W:\n", T1_refined)


### Aruco marker world frame calibration

#

In [ ]:
import cv2
import cv2.aruco as aruco
import numpy as np
import pickle

def solve_pnp(marker_corners, marker_length, K, dist):
    """Solve PnP for a single Aruco marker."""
    half_length = marker_length / 2

    # 3D points of the marker corners in the world coordinate system (origin at one corner)
    obj_points = np.array([
        [0, 0, 0],                  # Bottom-left (origin)
        [marker_length, 0, 0],      # Bottom-right (X-axis)
        [marker_length, marker_length, 0], # Top-right
        [0, marker_length, 0]       # Top-left (Z-axis)
    ], dtype=np.float32)
    print("Object points:\n", obj_points)
    # Solve PnP to find rotation and translation
    ret, rvec, tvec = cv2.solvePnP(obj_points, marker_corners, K, dist)
    if ret:
        R, _ = cv2.Rodrigues(rvec)
        return R, tvec
    return None, None

def draw_axes(image, R, T, K, dist):
    """Draw the world frame origin and axes on the image."""
    axis_length = 0.05  # 5 cm

    # Define the 3D points for the axes
    axis_points = np.float32([
        [0, 0, 0],              # Origin
        [axis_length, 0, 0],    # X-axis (red)
        [0, axis_length, 0],    # Y-axis (green)
        [0, 0, axis_length]     # Z-axis (blue)
    ]).reshape(-1, 3)

    # Project the 3D axis points onto the image plane
    img_points, _ = cv2.projectPoints(axis_points, cv2.Rodrigues(R)[0], T, K, dist)

    # Draw the origin point
    origin = tuple(img_points[0].ravel().astype(int))
    cv2.circle(image, origin, 5, (0, 0, 255), -1)

    # Draw the X, Y, Z axes
    cv2.line(image, origin, tuple(img_points[1].ravel().astype(int)), (0, 0, 255), 3)  # X-axis (red)
    cv2.line(image, origin, tuple(img_points[2].ravel().astype(int)), (0, 255, 0), 3)  # Y-axis (green)
    cv2.line(image, origin, tuple(img_points[3].ravel().astype(int)), (255, 0, 0), 3)  # Z-axis (blue)

    # Annotate the axes
    cv2.putText(image, "X", tuple(img_points[1].ravel().astype(int)), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 255), 2)
    cv2.putText(image, "Y", tuple(img_points[2].ravel().astype(int)), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2)
    cv2.putText(image, "Z", tuple(img_points[3].ravel().astype(int)), cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 0, 0), 2)


# Load intrinsic calibration data
# # Load intrinsic parameters
calib_data_file = f"../camera_calibration/05-08-25/camera_calib_data.pkl"
with open(calib_data_file, 'rb') as f:
    camera_calib_data = pickle.load(f)

K = camera_calib_data["cam0"]["intrinsics"]["K"]
dist = camera_calib_data["cam0"]["intrinsics"]["d"]


# Aruco marker parameters
aruco_dict = aruco.getPredefinedDictionary(aruco.DICT_4X4_50)
marker_length = 0.004  # Length of the marker side in meters

# Initialize the Aruco detector
detector = aruco.ArucoDetector(aruco_dict)

# Load image
img = cv2.imread(f"../camera_calibration/05-08-25/extrinsic_calib_images/cam0/img_aruco.png")
img_gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)

# Detect the Aruco marker
corners, ids, _ = detector.detectMarkers(img_gray)
print(corners)

if ids is not None:
    R_0_w, T_0_w = solve_pnp(corners[0], marker_length, K, dist)
    if R_0_w is not None:
        print(f"Camera 0 to World Rotation:\n{R_0_w}")
        print(f"Camera 0 to World Translation:\n{T_0_w}")

        # Draw the marker and the world frame
        aruco.drawDetectedMarkers(img, corners, ids)
        draw_axes(img, R_0_w, T_0_w, K, dist)

# Compute Camera 1 to world transform using stereo extrinsics
R_1_0 = camera_calib_data["stereo_extrinsics"]["R_1_0"]
T_1_0 = camera_calib_data["stereo_extrinsics"]["T_1_0"]
R_1_w = np.dot(R_1_0, R_0_w)
T_1_w = np.dot(R_1_0, T_0_w) + T_1_0
print(f"Camera 1 to World Rotation:\n{R_1_w}")
print(f"Camera 1 to World Translation:\n{T_1_w}")

# save all calibration data
camera_calib_data["cam0"]["extrinsics"]["R"] = R_0_w
camera_calib_data["cam0"]["extrinsics"]["T"] = T_0_w
camera_calib_data["cam1"]["extrinsics"]["R"] = R_1_w
camera_calib_data["cam1"]["extrinsics"]["T"] = T_1_w

# with open(f"{calib_base_folder}/camera_calib_data.pkl", "wb") as f:
#     pickle.dump(camera_calib_data, f)

cv2.imshow("Camera 0 - Aruco Marker with World Frame", img)
# cv2.imwrite(f"{calib_base_folder}/extrinsic_calib_images/cam0/img_aruco_with_axes.png", img)
cv2.waitKey(0)
cv2.destroyAllWindows()

In [ ]:
# Verify camera to world transform by projecting axes onto image
import cv2
import numpy as np
import cv2.aruco as aruco
import pickle
import os

# Load calibration data
with open(f"../camera_calibration/05-08-25/camera_calib_data.pkl", "rb") as f:
    camera_calib_data = pickle.load(f)

# Load intrinsic and extrinsic parameters for both cameras
K0 = camera_calib_data["cam0"]["intrinsics"]["K"]
dist0 = camera_calib_data["cam0"]["intrinsics"]["d"]
R_0_w = camera_calib_data["cam0"]["extrinsics"]["R"]
T_0_w = camera_calib_data["cam0"]["extrinsics"]["T"]
print("Camera 0 to World Rotation:\n", R_0_w)
print("Camera 0 to World Translation:\n", T_0_w)

K1 = camera_calib_data["cam1"]["intrinsics"]["K"]
dist1 = camera_calib_data["cam1"]["intrinsics"]["d"]
R_1_w = camera_calib_data["cam1"]["extrinsics"]["R"]
T_1_w = camera_calib_data["cam1"]["extrinsics"]["T"]
print("Camera 1 to World Rotation:\n", R_1_w)
print("Camera 1 to World Translation:\n", T_1_w)

# Load images for verification
img0 = cv2.imread(f"../tip_pose_images/cam0_0.png")
img1 = cv2.imread(f"../tip_pose_images/cam1_0.png")

# Draw world frame axes on Camera 0 image
draw_axes(img0, R_0_w, T_0_w, K0, dist0)
cv2.imshow("Camera 0 - World Frame Projection", img0)

# Draw world frame axes on Camera 1 image
draw_axes(img1, R_1_w, T_1_w, K1, dist1)
cv2.imshow("Camera 1 - World Frame Projection", img1)

cv2.waitKey(0)
cv2.destroyAllWindows()

# Test

In [ ]:
import pickle
import numpy as np

with open("../camera_calibration/05-08-25/camera_calib_data.pkl", "rb") as f:
    camera_calib = pickle.load(f)

K0 = camera_calib["cam0"]["intrinsics"]["K"]
dist0 = camera_calib["cam0"]["intrinsics"]["d"]
R_1_0 = camera_calib["stereo_extrinsics"]["R_1_0"]
T_1_0 = camera_calib["stereo_extrinsics"]["T_1_0"]

img_points = np.array([
    [317., 117.],
    [318., 144.],
    [316., 199.],
    [308., 118.],
    # [278, 148],
    # [326, 118],
    # [354, 150]
], dtype=np.float32)
obj_points = 0.001 * np.array([[0, 4.125, 0],
                [0, 16, 9],
                [0, 0, 23.95],
                [-4.125, 0, 0],
                # [-16, 0, 9],
                # [4.125, 0, 0],
                # [16, 0, 9]
                ])

ret, rvec, tvec, _ = cv2.solvePnPRansac(obj_points, img_points, K0, dist0, flags=cv2.SOLVEPNP_P3P)
R0 = cv2.Rodrigues(rvec)[0]
T0 = tvec
R1 = np.dot(R_1_0, R0)
T1 = np.dot(R_1_0, T0) + T_1_0
print("R1:\n", R1)
print("T1:\n", T1)

# Update and save camera calibration data
camera_calib["cam0"]["extrinsics"]["R"] = R0
camera_calib["cam0"]["extrinsics"]["T"] = T0
camera_calib["cam1"]["extrinsics"]["R"] = R1
camera_calib["cam1"]["extrinsics"]["T"] = T1

with open(f"../camera_calibration/05-16-25/camera_calib_data.pkl", "wb") as f:
    pickle.dump(camera_calib, f)

In [ ]:
# Visualize new transform
import cv2
import numpy as np
import pickle

def draw_axes(image, R, T, K, dist):
    """Draw the world frame origin and axes on the image."""
    axis_length = 0.05  # 5 cm

    # Define the 3D points for the axes
    axis_points = np.float32([
        [0, 0, 0],              # Origin
        [axis_length, 0, 0],    # X-axis (red)
        [0, axis_length, 0],    # Y-axis (green)
        [0, 0, axis_length]     # Z-axis (blue)
    ]).reshape(-1, 3)

    # Project the 3D axis points onto the image plane
    img_points, _ = cv2.projectPoints(axis_points, cv2.Rodrigues(R)[0], T, K, dist)

    # Draw the origin point
    origin = tuple(img_points[0].ravel().astype(int))
    cv2.circle(image, origin, 5, (0, 0, 255), -1)

    # Draw the X, Y, Z axes
    cv2.line(image, origin, tuple(img_points[1].ravel().astype(int)), (0, 0, 255), 3)  # X-axis (red)
    cv2.line(image, origin, tuple(img_points[2].ravel().astype(int)), (0, 255, 0), 3)  # Y-axis (green)
    cv2.line(image, origin, tuple(img_points[3].ravel().astype(int)), (255, 0, 0), 3)  # Z-axis (blue)

    # Annotate the axes
    cv2.putText(image, "X", tuple(img_points[1].ravel().astype(int)), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 255), 2)
    cv2.putText(image, "Y", tuple(img_points[2].ravel().astype(int)), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2)
    cv2.putText(image, "Z", tuple(img_points[3].ravel().astype(int)), cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 0, 0), 2)

with open(f"../camera_calibration/05-16-25/camera_calib_data.pkl", "rb") as f:
    camera_calib = pickle.load(f)

K0 = camera_calib["cam0"]["intrinsics"]["K"]
dist0 = camera_calib["cam0"]["intrinsics"]["d"]
R0 = camera_calib["cam0"]["extrinsics"]["R"]
T0 = camera_calib["cam0"]["extrinsics"]["T"]
K1 = camera_calib["cam1"]["intrinsics"]["K"]
dist1 = camera_calib["cam1"]["intrinsics"]["d"]
R1 = camera_calib["cam1"]["extrinsics"]["R"]
T1 = camera_calib["cam1"]["extrinsics"]["T"]

# Load images for verification
img0 = cv2.imread("../camera_calibration/05-16-25/extrinsic_calib_images/cam0_0.png")
img1 = cv2.imread("../camera_calibration/05-16-25/extrinsic_calib_images/cam1_0.png")
# Downsize images for display
img0 = cv2.resize(img0, (640, 480))
img1 = cv2.resize(img1, (640, 480))

# Draw world frame axes on Camera 0 image
draw_axes(img0, R0, T0, K0, dist0)
cv2.imshow("Camera 0 - World Frame Projection", img0)
# Draw world frame axes on Camera 1 image
draw_axes(img1, R1, T1, K1, dist1)
cv2.imshow("Camera 1 - World Frame Projection", img1)
cv2.waitKey(0)
cv2.destroyAllWindows()

# Write calibration matrices to python module

In [ ]:
import numpy as np
import pickle

# # Load camera calibration data from pkl file
# with open(f"{calib_base_folder}/camera_calib_data.pkl", "rb") as f:
#     camera_calib_data = pickle.load(f)

# Write to a python module
calib_module_path = f"{calib_base_folder}/camera_calib_data.py"
with open(calib_module_path, "w") as f:
    f.write("# This file is auto-generated from camera calibration data.\n")
    f.write("import numpy as np\n\n")
    f.write(f"K0 = np.{repr(camera_calib_data['cam0']['intrinsics']['K'])}\n")
    f.write(f"d0 = np.{repr(camera_calib_data['cam0']['intrinsics']['d'])}\n")
    f.write(f"K1 = np.{repr(camera_calib_data['cam1']['intrinsics']['K'])}\n")
    f.write(f"d1 = np.{repr(camera_calib_data['cam1']['intrinsics']['d'])}\n")
    f.write(f"K2 = np.{repr(camera_calib_data['cam2']['intrinsics']['K'])}\n")
    f.write(f"d2 = np.{repr(camera_calib_data['cam2']['intrinsics']['d'])}\n")
    f.write(f"R_1_0 = np.{repr(camera_calib_data['stereo_extrinsics']['R_1_0'])}\n")
    f.write(f"T_1_0 = np.{repr(camera_calib_data['stereo_extrinsics']['T_1_0'])}\n")
    f.write(f"R_0_w = np.{repr(camera_calib_data['cam0']['extrinsics']['R'])}\n")
    f.write(f"T_0_w = np.{repr(camera_calib_data['cam0']['extrinsics']['T'])}\n")
    f.write(f"R_1_w = np.{repr(camera_calib_data['cam1']['extrinsics']['R'])}\n")
    f.write(f"T_1_w = np.{repr(camera_calib_data['cam1']['extrinsics']['T'])}\n")
    f.write(f"R_2_1 = np.{repr(camera_calib_data['stereo_extrinsics']['R_2_1'])}\n")
    f.write(f"T_2_1 = np.{repr(camera_calib_data['stereo_extrinsics']['T_2_1'])}\n")
    f.write(f"R_2_w = np.{repr(camera_calib_data['cam2']['extrinsics']['R'])}\n")
    f.write(f"T_2_w = np.{repr(camera_calib_data['cam2']['extrinsics']['T'])}\n")
